# Caribbean Beach Habitat
Created by Blair Tirpak. Last run July 17, 2023
Updated by Blair Tirpak August 30, 2023

In [1]:
# Import system modules
import arcpy
from arcpy import env
from arcpy.sa import *

# Set environment settings
env.workspace = r"C:\Data\Indicators\Beach\Beach.gdb"
workspace = r"C:\Data\Indicators\Beach\Beach.gdb"

# define spatial reference
sr= arcpy.SpatialReference(5070)

# Final Extent
extent = r"C:\Data\Indicators\VIPR_Extent_v6.tif"
# define final indicator outputs
VIPRout = r"C:\Data\Indicators\Beach\CaribbeanBeachHabitat.tif"

# Beach 

In [32]:
# beach from OSM for USVI
OSM_b = r"C:\Data\Indicators\Beach\PR\beach20230628.shp"

# bufer beaches by 15m 
arcpy.analysis.PairwiseBuffer(OSM_b, "OSM_b15", "15 Meters", "ALL", None, "PLANAR", "0 Meters")

arcpy.management.Project("OSM_b15", "OSM_b15_albers", sr, "WGS_1984_(ITRF00)_To_NAD_1983", 'GEOGCS["GCS_WGS_1984",DATUM["D_WGS_1984",SPHEROID["WGS_1984",6378137.0,298.257223563]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]]', "NO_PRESERVE_SHAPE", None, "NO_VERTICAL")

with arcpy.EnvManager(snapRaster=extent, cellSize=extent):
    arcpy.conversion.PolygonToRaster("OSM_b15_albers", "OBJECTID", "OSM_b15_albersR", "CELL_CENTER", "NONE", 30, "BUILD")

## PR Beach

In [33]:
with arcpy.EnvManager(snapRaster=extent, cellSize=extent):
    arcpy.conversion.PolygonToRaster("OSM_b15_albers", "OBJECTID", "OSM_b15_albersR", "CELL_CENTER", "NONE", 30, "BUILD")
    

In [36]:
# PR GAP
PRgap = r"C:\Data\Indicators\Beach\PR\pr_landcover\prgap_landcover.img"
PR_gap_beach = arcpy.sa.Reclassify(PRgap, "PRGAP_CL", "'Gravel beaches and stony shoreline' 1;'Fine to coarse sandy beaches, mixed sand and gravel beaches' 1", "NODATA");
PR_gap_beach.save("PR_gap_beach")

In [37]:
# resample to 30 m
with arcpy.EnvManager(scratchWorkspace=workspace, outputCoordinateSystem=sr, snapRaster=extent, workspace=workspace):
    arcpy.management.Resample("PR_gap_beach", "PR_gap_beach_30", "30 30", "NEAREST")


In [38]:
# merge two beach files together
arcpy.management.MosaicToNewRaster("PR_gap_beach_30;OSM_b15_albersR", workspace, "PR_Beach_all", sr, "8_BIT_UNSIGNED", None, 1, "MAXIMUM", "FIRST")

<Result 'C:\\Data\\Indicators\\Beach\\Beach.gdb\\PR_Beach_all'>

## VI

In [45]:
# combine Beach polys from Joe for USVI
STJ = r"C:\Data\Indicators\Beach\InputDataFromJoeDwyer3-3-2023\STJ_Beaches.shp"
STT = r"C:\Data\Indicators\Beach\InputDataFromJoeDwyer3-3-2023\STT_Beaches.shp"                            
STX = r"C:\Data\Indicators\Beach\InputDataFromJoeDwyer3-3-2023\STXBeaches.shp"  

arcpy.Merge_management([STJ,STT,STX], "VI_Beaches_Joe", "NO_SOURCE_INFO")

<Result 'C:\\Data\\Indicators\\Beach\\Beach.gdb\\VI_Beaches_Joe'>

In [47]:
# bufer beaches by 15m 
arcpy.analysis.PairwiseBuffer("VI_Beaches_Joe", "VI_Beaches_Joe15", "15 Meters", "ALL", None, "PLANAR", "0 Meters")
arcpy.Project_management("VI_Beaches_Joe15", "VI_Beaches_Joe15_Albers", sr, "WGS_1984_(ITRF00)_To_NAD_1983", 
                         'PROJCS["WGS_1984_UTM_Zone_20N",GEOGCS["GCS_WGS_1984",DATUM["D_WGS_1984",SPHEROID["WGS_1984",6378137.0,298.257223563]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]],PROJECTION["Transverse_Mercator"],PARAMETER["False_Easting",500000.0],PARAMETER["False_Northing",0.0],PARAMETER["Central_Meridian",-63.0],PARAMETER["Scale_Factor",0.9996],PARAMETER["Latitude_Of_Origin",0.0],UNIT["Meter",1.0]]', "NO_PRESERVE_SHAPE", None, "NO_VERTICAL")
with arcpy.EnvManager(snapRaster=extent, cellSize=extent):
    arcpy.conversion.PolygonToRaster("VI_Beaches_Joe15_Albers", "OBJECTID", "VI_Beaches_JoeR", "CELL_CENTER", "NONE", 30, "BUILD")

In [50]:
# gap data
# extract beach
GAP = r"C:\Data\Indicators\Beach\USVI_GAP_DVD\GISdata\Landcover\1_Land_cover_grid\USVI_Land_Cover_USVIGAP.img"
GAP_beach = arcpy.sa.Reclassify(GAP, "Class_Name", "'Fine to Medium Grained Sandy Beaches' 1;'Gravel Beaches' 1;'Mixed Sand and Gravel Beaches' 1", "NODATA")
GAP_beach.save("GAP_beach")
# resample to 30 m
with arcpy.EnvManager(scratchWorkspace=workspace, outputCoordinateSystem=sr, snapRaster=extent, workspace=workspace):
    arcpy.management.Resample("GAP_beach", "GAP_beach_30", "30 30", "NEAREST")
    
# merge two beach files together
arcpy.management.MosaicToNewRaster("GAP_beach_30;VI_Beaches_JoeR", workspace, "VI_Beach_all", sr, "8_BIT_UNSIGNED", None, 1, "MAXIMUM", "FIRST")

<Result 'C:\\Data\\Indicators\\Beach\\Beach.gdb\\VI_Beach_all'>

## Merge PR and VI 

In [51]:
# beach
with arcpy.EnvManager(outputCoordinateSystem=sr, snapRaster=extent, cellSize=extent):
    arcpy.MosaicToNewRaster_management("PR_Beach_all;VI_Beach_all", env.workspace, "VIPR_beach", sr, 
                                   "8_BIT_UNSIGNED", "30", "1", "Maximum","FIRST")

# Species

## PR Species

In [11]:
# Set environment settings
sp = r"C:\Data\Indicators\Beach\PR\Predicted_habitat\1_Raster_files\Charadrius_wilsonia_habitat.img"

# extract predicted only
PRwilsonia = Reclassify(sp, "Value", RemapRange([[0,1,"NODATA"],[2,2,1]]))
PRwilsonia.save("PRwilsonia")

# resample to 30 m
with arcpy.EnvManager(scratchWorkspace=workspace, outputCoordinateSystem=sr, snapRaster=extent, workspace=workspace):
    arcpy.management.Resample("PRwilsonia", "PRwilsonia30", "30 30", "NEAREST")

### turtles

In [12]:
#SWOT points
Tpt = r"C:\Data\Indicators\Beach\turtles\obis_seamap_swot_site_locations_shapefile_78020\obis_seamap_swot_64a6062cbee32_20230705_204220_site_locations_shapefile.shp"


In [15]:
# SWOT points for Hawksbill 
arcpy.conversion.FeatureClassToFeatureClass(Tpt, workspace, "Hawskbill_PR", "country = 'Puerto Rico' And commonname = 'Hawksbill sea turtle'", 'Id "Id" true true false 6 Long 0 6,First,#,obis_seamap_swot_64a6062cbee32_20230705_204220_site_locations_shapefile,Id,-1,-1;code "code" true true false 10 Text 0 0,First,#,obis_seamap_swot_64a6062cbee32_20230705_204220_site_locations_shapefile,code,0,10;country "country" true true false 60 Text 0 0,First,#,obis_seamap_swot_64a6062cbee32_20230705_204220_site_locations_shapefile,country,0,60;siteid "siteid" true true false 20 Text 0 0,First,#,obis_seamap_swot_64a6062cbee32_20230705_204220_site_locations_shapefile,siteid,0,20;site_name "site_name" true true false 180 Text 0 0,First,#,obis_seamap_swot_64a6062cbee32_20230705_204220_site_locations_shapefile,site_name,0,180;compiler "compiler" true true false 100 Text 0 0,First,#,obis_seamap_swot_64a6062cbee32_20230705_204220_site_locations_shapefile,compiler,0,100;species "species" true true false 25 Text 0 0,First,#,obis_seamap_swot_64a6062cbee32_20230705_204220_site_locations_shapefile,species,0,25;commonname "commonname" true true false 30 Text 0 0,First,#,obis_seamap_swot_64a6062cbee32_20230705_204220_site_locations_shapefile,commonname,0,30;years_mon "years_mon" true true false 254 Text 0 0,First,#,obis_seamap_swot_64a6062cbee32_20230705_204220_site_locations_shapefile,years_mon,0,254;monitoring "monitoring" true true false 150 Text 0 0,First,#,obis_seamap_swot_64a6062cbee32_20230705_204220_site_locations_shapefile,monitoring,0,150;latitude "latitude" true true false 12 Double 6 11,First,#,obis_seamap_swot_64a6062cbee32_20230705_204220_site_locations_shapefile,latitude,-1,-1;longitude "longitude" true true false 12 Double 6 11,First,#,obis_seamap_swot_64a6062cbee32_20230705_204220_site_locations_shapefile,longitude,-1,-1;neststatus "neststatus" true true false 15 Text 0 0,First,#,obis_seamap_swot_64a6062cbee32_20230705_204220_site_locations_shapefile,neststatus,0,15;contact1 "contact1" true true false 254 Text 0 0,First,#,obis_seamap_swot_64a6062cbee32_20230705_204220_site_locations_shapefile,contact1,0,254;contact2 "contact2" true true false 254 Text 0 0,First,#,obis_seamap_swot_64a6062cbee32_20230705_204220_site_locations_shapefile,contact2,0,254;contact3 "contact3" true true false 254 Text 0 0,First,#,obis_seamap_swot_64a6062cbee32_20230705_204220_site_locations_shapefile,contact3,0,254;contact4 "contact4" true true false 254 Text 0 0,First,#,obis_seamap_swot_64a6062cbee32_20230705_204220_site_locations_shapefile,contact4,0,254;contact5 "contact5" true true false 254 Text 0 0,First,#,obis_seamap_swot_64a6062cbee32_20230705_204220_site_locations_shapefile,contact5,0,254', '')

with arcpy.EnvManager(outputCoordinateSystem=sr, snapRaster=extent, cellSize=extent):
    arcpy.conversion.PointToRaster("Hawskbill_PR", "species", "Hawksbill_PR_30", "MOST_FREQUENT", "NONE", 30, "BUILD")


In [16]:
# Leatherback
   
# SWOT points 
arcpy.conversion.FeatureClassToFeatureClass(Tpt, workspace, "Leatherback_PR", "country = 'Puerto Rico' And commonname = 'Leatherback sea turtle'", 'Id "Id" true true false 6 Long 0 6,First,#,obis_seamap_swot_64a6062cbee32_20230705_204220_site_locations_shapefile,Id,-1,-1;code "code" true true false 10 Text 0 0,First,#,obis_seamap_swot_64a6062cbee32_20230705_204220_site_locations_shapefile,code,0,10;country "country" true true false 60 Text 0 0,First,#,obis_seamap_swot_64a6062cbee32_20230705_204220_site_locations_shapefile,country,0,60;siteid "siteid" true true false 20 Text 0 0,First,#,obis_seamap_swot_64a6062cbee32_20230705_204220_site_locations_shapefile,siteid,0,20;site_name "site_name" true true false 180 Text 0 0,First,#,obis_seamap_swot_64a6062cbee32_20230705_204220_site_locations_shapefile,site_name,0,180;compiler "compiler" true true false 100 Text 0 0,First,#,obis_seamap_swot_64a6062cbee32_20230705_204220_site_locations_shapefile,compiler,0,100;species "species" true true false 25 Text 0 0,First,#,obis_seamap_swot_64a6062cbee32_20230705_204220_site_locations_shapefile,species,0,25;commonname "commonname" true true false 30 Text 0 0,First,#,obis_seamap_swot_64a6062cbee32_20230705_204220_site_locations_shapefile,commonname,0,30;years_mon "years_mon" true true false 254 Text 0 0,First,#,obis_seamap_swot_64a6062cbee32_20230705_204220_site_locations_shapefile,years_mon,0,254;monitoring "monitoring" true true false 150 Text 0 0,First,#,obis_seamap_swot_64a6062cbee32_20230705_204220_site_locations_shapefile,monitoring,0,150;latitude "latitude" true true false 12 Double 6 11,First,#,obis_seamap_swot_64a6062cbee32_20230705_204220_site_locations_shapefile,latitude,-1,-1;longitude "longitude" true true false 12 Double 6 11,First,#,obis_seamap_swot_64a6062cbee32_20230705_204220_site_locations_shapefile,longitude,-1,-1;neststatus "neststatus" true true false 15 Text 0 0,First,#,obis_seamap_swot_64a6062cbee32_20230705_204220_site_locations_shapefile,neststatus,0,15;contact1 "contact1" true true false 254 Text 0 0,First,#,obis_seamap_swot_64a6062cbee32_20230705_204220_site_locations_shapefile,contact1,0,254;contact2 "contact2" true true false 254 Text 0 0,First,#,obis_seamap_swot_64a6062cbee32_20230705_204220_site_locations_shapefile,contact2,0,254;contact3 "contact3" true true false 254 Text 0 0,First,#,obis_seamap_swot_64a6062cbee32_20230705_204220_site_locations_shapefile,contact3,0,254;contact4 "contact4" true true false 254 Text 0 0,First,#,obis_seamap_swot_64a6062cbee32_20230705_204220_site_locations_shapefile,contact4,0,254;contact5 "contact5" true true false 254 Text 0 0,First,#,obis_seamap_swot_64a6062cbee32_20230705_204220_site_locations_shapefile,contact5,0,254', '')

with arcpy.EnvManager(outputCoordinateSystem=sr, snapRaster=extent, cellSize=extent):
    arcpy.conversion.PointToRaster("Leatherback_PR", "species", "Leatherback_PR_30", "MOST_FREQUENT", "NONE", 30, "BUILD")


In [17]:
# Green Sea Turtle
#points from SWOT 
arcpy.conversion.FeatureClassToFeatureClass(Tpt, workspace, "Green_PR", "country = 'Puerto Rico' And commonname = 'Green sea turtle'", 'Id "Id" true true false 6 Long 0 6,First,#,obis_seamap_swot_64a6062cbee32_20230705_204220_site_locations_shapefile,Id,-1,-1;code "code" true true false 10 Text 0 0,First,#,obis_seamap_swot_64a6062cbee32_20230705_204220_site_locations_shapefile,code,0,10;country "country" true true false 60 Text 0 0,First,#,obis_seamap_swot_64a6062cbee32_20230705_204220_site_locations_shapefile,country,0,60;siteid "siteid" true true false 20 Text 0 0,First,#,obis_seamap_swot_64a6062cbee32_20230705_204220_site_locations_shapefile,siteid,0,20;site_name "site_name" true true false 180 Text 0 0,First,#,obis_seamap_swot_64a6062cbee32_20230705_204220_site_locations_shapefile,site_name,0,180;compiler "compiler" true true false 100 Text 0 0,First,#,obis_seamap_swot_64a6062cbee32_20230705_204220_site_locations_shapefile,compiler,0,100;species "species" true true false 25 Text 0 0,First,#,obis_seamap_swot_64a6062cbee32_20230705_204220_site_locations_shapefile,species,0,25;commonname "commonname" true true false 30 Text 0 0,First,#,obis_seamap_swot_64a6062cbee32_20230705_204220_site_locations_shapefile,commonname,0,30;years_mon "years_mon" true true false 254 Text 0 0,First,#,obis_seamap_swot_64a6062cbee32_20230705_204220_site_locations_shapefile,years_mon,0,254;monitoring "monitoring" true true false 150 Text 0 0,First,#,obis_seamap_swot_64a6062cbee32_20230705_204220_site_locations_shapefile,monitoring,0,150;latitude "latitude" true true false 12 Double 6 11,First,#,obis_seamap_swot_64a6062cbee32_20230705_204220_site_locations_shapefile,latitude,-1,-1;longitude "longitude" true true false 12 Double 6 11,First,#,obis_seamap_swot_64a6062cbee32_20230705_204220_site_locations_shapefile,longitude,-1,-1;neststatus "neststatus" true true false 15 Text 0 0,First,#,obis_seamap_swot_64a6062cbee32_20230705_204220_site_locations_shapefile,neststatus,0,15;contact1 "contact1" true true false 254 Text 0 0,First,#,obis_seamap_swot_64a6062cbee32_20230705_204220_site_locations_shapefile,contact1,0,254;contact2 "contact2" true true false 254 Text 0 0,First,#,obis_seamap_swot_64a6062cbee32_20230705_204220_site_locations_shapefile,contact2,0,254;contact3 "contact3" true true false 254 Text 0 0,First,#,obis_seamap_swot_64a6062cbee32_20230705_204220_site_locations_shapefile,contact3,0,254;contact4 "contact4" true true false 254 Text 0 0,First,#,obis_seamap_swot_64a6062cbee32_20230705_204220_site_locations_shapefile,contact4,0,254;contact5 "contact5" true true false 254 Text 0 0,First,#,obis_seamap_swot_64a6062cbee32_20230705_204220_site_locations_shapefile,contact5,0,254', '')

with arcpy.EnvManager(outputCoordinateSystem=sr, snapRaster=extent, cellSize=extent):
    arcpy.conversion.PointToRaster("Green_PR", "species", "Green_PR_30", "MOST_FREQUENT", "NONE", 30, "BUILD")


### Make spp presence file

In [18]:
# merge spp rasters
with arcpy.EnvManager(outputCoordinateSystem=sr, snapRaster=extent, cellSize=extent):
    arcpy.MosaicToNewRaster_management("PRwilsonia30;Hawksbill_PR_30;Green_PR_30;Leatherback_PR_30", env.workspace, "PR_sp_presence", sr, 
                                   "8_BIT_UNSIGNED", "30", "1", "MAXIMUM","FIRST")

In [19]:
# Clip to beach
PR_sp_beach = arcpy.sa.ExtractByMask("PR_sp_presence", "PR_Beach_all"); 
PR_sp_beach.save("PR_sp_beach")

## VI Spcies

### birds

In [20]:
# Wilson's Plover
sp = r"C:\Data\Indicators\Beach\USVI_GAP_DVD\GISdata\Vertebrates\Predicted_habitat\1_Raster_files\Charadrius_wilsonia_PHM.img"

# extract predicted only
VIwilsonia = Reclassify(sp, "Value", RemapRange([[0,1,"NODATA"],[2,2,1]]))
VIwilsonia.save("VIwilsonia")

# resample to 30 m
with arcpy.EnvManager(scratchWorkspace=workspace, outputCoordinateSystem=sr, snapRaster=extent, workspace=workspace):
    arcpy.management.Resample("VIwilsonia", "VIwilsonia30", "30 30", "NEAREST")

In [21]:
# American Oystercatcher
sp = r"C:\Data\Indicators\Beach\USVI_GAP_DVD\GISdata\Vertebrates\Predicted_habitat\1_Raster_files\Haematopus_palliatus_PHM.img"

# extract predicted only
VIpalliatus = Reclassify(sp, "Value", RemapRange([[0,1,"NODATA"],[2,2,1]]))
VIpalliatus.save("VIpalliatus")

# resample to 30 m
with arcpy.EnvManager(scratchWorkspace=workspace, outputCoordinateSystem=sr, snapRaster=extent, workspace=workspace):
    arcpy.management.Resample("VIpalliatus", "VIpalliatus30", "30 30", "NEAREST")

### Turtles

In [22]:
# Hawksbill
# Set environment settings
sp = r"C:\Data\Indicators\Beach\USVI_GAP_DVD\GISdata\Vertebrates\Predicted_habitat\1_Raster_files\Eretmochelys_imbricata_PHM.img"

# extract predicted only
VIimbricata = Reclassify(sp, "Value", RemapRange([[0,1,"NODATA"],[2,2,1]]))
VIimbricata.save("VIimbricata")

# resample to 30 m
with arcpy.EnvManager(scratchWorkspace=workspace, outputCoordinateSystem=sr, snapRaster=extent, workspace=workspace):
    arcpy.management.Resample("VIimbricata", "VIimbricata30", "30 30", "NEAREST")

#SWOT points
Tpt = r"C:\Data\Indicators\Beach\turtles\obis_seamap_swot_site_locations_shapefile_78020\obis_seamap_swot_64a6062cbee32_20230705_204220_site_locations_shapefile.shp"

# SWOT points for Hawksbill
arcpy.conversion.FeatureClassToFeatureClass(Tpt, workspace, "VIHawskbill_VI", "country = 'Virgin Islands, United States' And commonname = 'Hawksbill sea turtle' And siteid <> 'VI05'")

with arcpy.EnvManager(outputCoordinateSystem=sr, snapRaster=extent, cellSize=extent):
    arcpy.conversion.PointToRaster("VIHawskbill_VI", "species", "VIHawksbill_VI_30", "MOST_FREQUENT", "NONE", 30, "BUILD")

# merge turtle files together
arcpy.management.MosaicToNewRaster("VIimbricata30;VIHawksbill_VI_30", workspace, "VIHawksbill_all", sr, "8_BIT_UNSIGNED", None, 1, "MAXIMUM", "FIRST")

<Result 'C:\\Data\\Indicators\\Beach\\Beach.gdb\\VIHawksbill_all'>

In [25]:
# Leatherback
# Set environment settings
sp = r"C:\Data\Indicators\Beach\USVI_GAP_DVD\GISdata\Vertebrates\Predicted_habitat\1_Raster_files\Dermochelys_coriacea_PHM.img"

# extract predicted only
VIcoriacea = Reclassify(sp, "Value", RemapRange([[0,1,"NODATA"],[2,2,1]]))
VIcoriacea.save("VIcoriacea")

# resample to 30 m
with arcpy.EnvManager(scratchWorkspace=workspace, outputCoordinateSystem=sr, snapRaster=extent, workspace=workspace):
    arcpy.management.Resample("VIcoriacea", "VIcoriacea30", "30 30", "NEAREST")
    
# SWOT points for Hawksbill that don't overlap
arcpy.conversion.FeatureClassToFeatureClass(Tpt, workspace, "Leatherback_VI", "country = 'Virgin Islands, United States' And commonname = 'Leatherback sea turtle'", 'Id "Id" true true false 6 Long 0 6,First,#,obis_seamap_swot_64a6062cbee32_20230705_204220_site_locations_shapefile,Id,-1,-1;code "code" true true false 10 Text 0 0,First,#,obis_seamap_swot_64a6062cbee32_20230705_204220_site_locations_shapefile,code,0,10;country "country" true true false 60 Text 0 0,First,#,obis_seamap_swot_64a6062cbee32_20230705_204220_site_locations_shapefile,country,0,60;siteid "siteid" true true false 20 Text 0 0,First,#,obis_seamap_swot_64a6062cbee32_20230705_204220_site_locations_shapefile,siteid,0,20;site_name "site_name" true true false 180 Text 0 0,First,#,obis_seamap_swot_64a6062cbee32_20230705_204220_site_locations_shapefile,site_name,0,180;compiler "compiler" true true false 100 Text 0 0,First,#,obis_seamap_swot_64a6062cbee32_20230705_204220_site_locations_shapefile,compiler,0,100;species "species" true true false 25 Text 0 0,First,#,obis_seamap_swot_64a6062cbee32_20230705_204220_site_locations_shapefile,species,0,25;commonname "commonname" true true false 30 Text 0 0,First,#,obis_seamap_swot_64a6062cbee32_20230705_204220_site_locations_shapefile,commonname,0,30;years_mon "years_mon" true true false 254 Text 0 0,First,#,obis_seamap_swot_64a6062cbee32_20230705_204220_site_locations_shapefile,years_mon,0,254;monitoring "monitoring" true true false 150 Text 0 0,First,#,obis_seamap_swot_64a6062cbee32_20230705_204220_site_locations_shapefile,monitoring,0,150;latitude "latitude" true true false 12 Double 6 11,First,#,obis_seamap_swot_64a6062cbee32_20230705_204220_site_locations_shapefile,latitude,-1,-1;longitude "longitude" true true false 12 Double 6 11,First,#,obis_seamap_swot_64a6062cbee32_20230705_204220_site_locations_shapefile,longitude,-1,-1;neststatus "neststatus" true true false 15 Text 0 0,First,#,obis_seamap_swot_64a6062cbee32_20230705_204220_site_locations_shapefile,neststatus,0,15;contact1 "contact1" true true false 254 Text 0 0,First,#,obis_seamap_swot_64a6062cbee32_20230705_204220_site_locations_shapefile,contact1,0,254;contact2 "contact2" true true false 254 Text 0 0,First,#,obis_seamap_swot_64a6062cbee32_20230705_204220_site_locations_shapefile,contact2,0,254;contact3 "contact3" true true false 254 Text 0 0,First,#,obis_seamap_swot_64a6062cbee32_20230705_204220_site_locations_shapefile,contact3,0,254;contact4 "contact4" true true false 254 Text 0 0,First,#,obis_seamap_swot_64a6062cbee32_20230705_204220_site_locations_shapefile,contact4,0,254;contact5 "contact5" true true false 254 Text 0 0,First,#,obis_seamap_swot_64a6062cbee32_20230705_204220_site_locations_shapefile,contact5,0,254', '')

with arcpy.EnvManager(outputCoordinateSystem=sr, snapRaster=extent, cellSize=extent):
    arcpy.conversion.PointToRaster("Leatherback_VI", "species", "Leatherback_VI_30", "MOST_FREQUENT", "NONE", 30, "BUILD")

# merge turtle files together
arcpy.management.MosaicToNewRaster("VIcoriacea30;Leatherback_VI_30", workspace, "VILeatherback_all", sr, 
                                   "8_BIT_UNSIGNED", None, 1, "MAXIMUM", "FIRST")

<Result 'C:\\Data\\Indicators\\Beach\\Beach.gdb\\VILeatherback_all'>

In [26]:
# Green Sea Turtle
#points from SWOT overlap
# Set environment settings
sp = r"C:\Data\Indicators\Beach\USVI_GAP_DVD\GISdata\Vertebrates\Predicted_habitat\1_Raster_files\Chelonia_mydas_PHM.img"

# extract predicted only
VImydas = Reclassify(sp, "Value", RemapRange([[0,1,"NODATA"],[2,2,1]]))
VImydas.save("VImydas")

# resample to 30 m
with arcpy.EnvManager(scratchWorkspace=workspace, outputCoordinateSystem=sr, snapRaster=extent, workspace=workspace):
    arcpy.management.Resample("VImydas", "VImydas30", "30 30", "NEAREST")
    
# SWOT points for Hawksbill that don't overlap
arcpy.conversion.FeatureClassToFeatureClass(Tpt, workspace, "Green_VI", "country = 'Virgin Islands, United States' And commonname = 'Green sea turtle'", 'Id "Id" true true false 6 Long 0 6,First,#,obis_seamap_swot_64a6062cbee32_20230705_204220_site_locations_shapefile,Id,-1,-1;code "code" true true false 10 Text 0 0,First,#,obis_seamap_swot_64a6062cbee32_20230705_204220_site_locations_shapefile,code,0,10;country "country" true true false 60 Text 0 0,First,#,obis_seamap_swot_64a6062cbee32_20230705_204220_site_locations_shapefile,country,0,60;siteid "siteid" true true false 20 Text 0 0,First,#,obis_seamap_swot_64a6062cbee32_20230705_204220_site_locations_shapefile,siteid,0,20;site_name "site_name" true true false 180 Text 0 0,First,#,obis_seamap_swot_64a6062cbee32_20230705_204220_site_locations_shapefile,site_name,0,180;compiler "compiler" true true false 100 Text 0 0,First,#,obis_seamap_swot_64a6062cbee32_20230705_204220_site_locations_shapefile,compiler,0,100;species "species" true true false 25 Text 0 0,First,#,obis_seamap_swot_64a6062cbee32_20230705_204220_site_locations_shapefile,species,0,25;commonname "commonname" true true false 30 Text 0 0,First,#,obis_seamap_swot_64a6062cbee32_20230705_204220_site_locations_shapefile,commonname,0,30;years_mon "years_mon" true true false 254 Text 0 0,First,#,obis_seamap_swot_64a6062cbee32_20230705_204220_site_locations_shapefile,years_mon,0,254;monitoring "monitoring" true true false 150 Text 0 0,First,#,obis_seamap_swot_64a6062cbee32_20230705_204220_site_locations_shapefile,monitoring,0,150;latitude "latitude" true true false 12 Double 6 11,First,#,obis_seamap_swot_64a6062cbee32_20230705_204220_site_locations_shapefile,latitude,-1,-1;longitude "longitude" true true false 12 Double 6 11,First,#,obis_seamap_swot_64a6062cbee32_20230705_204220_site_locations_shapefile,longitude,-1,-1;neststatus "neststatus" true true false 15 Text 0 0,First,#,obis_seamap_swot_64a6062cbee32_20230705_204220_site_locations_shapefile,neststatus,0,15;contact1 "contact1" true true false 254 Text 0 0,First,#,obis_seamap_swot_64a6062cbee32_20230705_204220_site_locations_shapefile,contact1,0,254;contact2 "contact2" true true false 254 Text 0 0,First,#,obis_seamap_swot_64a6062cbee32_20230705_204220_site_locations_shapefile,contact2,0,254;contact3 "contact3" true true false 254 Text 0 0,First,#,obis_seamap_swot_64a6062cbee32_20230705_204220_site_locations_shapefile,contact3,0,254;contact4 "contact4" true true false 254 Text 0 0,First,#,obis_seamap_swot_64a6062cbee32_20230705_204220_site_locations_shapefile,contact4,0,254;contact5 "contact5" true true false 254 Text 0 0,First,#,obis_seamap_swot_64a6062cbee32_20230705_204220_site_locations_shapefile,contact5,0,254', '')

with arcpy.EnvManager(outputCoordinateSystem=sr, snapRaster=extent, cellSize=extent):
    arcpy.conversion.PointToRaster("Green_VI", "species", "Green_VI_30", "MOST_FREQUENT", "NONE", 30, "BUILD")

# merge turtle files together
arcpy.management.MosaicToNewRaster("VImydas30;Green_VI_30", workspace, "VIGreen_all", sr, "8_BIT_UNSIGNED", None, 1, "MAXIMUM", "FIRST")

<Result 'C:\\Data\\Indicators\\Beach\\Beach.gdb\\VIGreen_all'>

In [27]:
# Loggerhead
# Set environment settings
    
# SWOT points for Loggerhead
arcpy.conversion.FeatureClassToFeatureClass(Tpt, workspace, "Loggerhead_VI", "country = 'Virgin Islands, United States' And commonname = 'Loggerhead sea turtle'", 'Id "Id" true true false 6 Long 0 6,First,#,obis_seamap_swot_64a6062cbee32_20230705_204220_site_locations_shapefile,Id,-1,-1;code "code" true true false 10 Text 0 0,First,#,obis_seamap_swot_64a6062cbee32_20230705_204220_site_locations_shapefile,code,0,10;country "country" true true false 60 Text 0 0,First,#,obis_seamap_swot_64a6062cbee32_20230705_204220_site_locations_shapefile,country,0,60;siteid "siteid" true true false 20 Text 0 0,First,#,obis_seamap_swot_64a6062cbee32_20230705_204220_site_locations_shapefile,siteid,0,20;site_name "site_name" true true false 180 Text 0 0,First,#,obis_seamap_swot_64a6062cbee32_20230705_204220_site_locations_shapefile,site_name,0,180;compiler "compiler" true true false 100 Text 0 0,First,#,obis_seamap_swot_64a6062cbee32_20230705_204220_site_locations_shapefile,compiler,0,100;species "species" true true false 25 Text 0 0,First,#,obis_seamap_swot_64a6062cbee32_20230705_204220_site_locations_shapefile,species,0,25;commonname "commonname" true true false 30 Text 0 0,First,#,obis_seamap_swot_64a6062cbee32_20230705_204220_site_locations_shapefile,commonname,0,30;years_mon "years_mon" true true false 254 Text 0 0,First,#,obis_seamap_swot_64a6062cbee32_20230705_204220_site_locations_shapefile,years_mon,0,254;monitoring "monitoring" true true false 150 Text 0 0,First,#,obis_seamap_swot_64a6062cbee32_20230705_204220_site_locations_shapefile,monitoring,0,150;latitude "latitude" true true false 12 Double 6 11,First,#,obis_seamap_swot_64a6062cbee32_20230705_204220_site_locations_shapefile,latitude,-1,-1;longitude "longitude" true true false 12 Double 6 11,First,#,obis_seamap_swot_64a6062cbee32_20230705_204220_site_locations_shapefile,longitude,-1,-1;neststatus "neststatus" true true false 15 Text 0 0,First,#,obis_seamap_swot_64a6062cbee32_20230705_204220_site_locations_shapefile,neststatus,0,15;contact1 "contact1" true true false 254 Text 0 0,First,#,obis_seamap_swot_64a6062cbee32_20230705_204220_site_locations_shapefile,contact1,0,254;contact2 "contact2" true true false 254 Text 0 0,First,#,obis_seamap_swot_64a6062cbee32_20230705_204220_site_locations_shapefile,contact2,0,254;contact3 "contact3" true true false 254 Text 0 0,First,#,obis_seamap_swot_64a6062cbee32_20230705_204220_site_locations_shapefile,contact3,0,254;contact4 "contact4" true true false 254 Text 0 0,First,#,obis_seamap_swot_64a6062cbee32_20230705_204220_site_locations_shapefile,contact4,0,254;contact5 "contact5" true true false 254 Text 0 0,First,#,obis_seamap_swot_64a6062cbee32_20230705_204220_site_locations_shapefile,contact5,0,254', '')

with arcpy.EnvManager(outputCoordinateSystem=sr, snapRaster=extent, cellSize=extent):
    arcpy.conversion.PointToRaster("Loggerhead_VI", "species", "Loggerhead_VI_30", "MOST_FREQUENT", "NONE", 30, "BUILD")

with arcpy.EnvManager(outputCoordinateSystem=sr, snapRaster=extent):
    Loggerhead_all = arcpy.sa.Reclassify("Leatherback_VI_30", "species", "'Dermochelys coriacea' 1", "DATA"); 
    Loggerhead_all.save("Loggerhead_all")

In [28]:
# merge spp rasters
arcpy.MosaicToNewRaster_management("VIwilsonia;VIpalliatus;VIimbricata;VIcoriacea;VImydas;Loggerhead_all", env.workspace, "VI_sp", sr, 
                                   "8_BIT_UNSIGNED", "10", "1", "MAXIMUM","FIRST")

# Clip to beach
VI_sp_beach = arcpy.sa.ExtractByMask("VI_sp", "VIPR_Beach"); 
VI_sp_beach.save("VI_sp_beach")

# Merge PR and VI 

In [29]:
# Species 
with arcpy.EnvManager(outputCoordinateSystem=sr, snapRaster=extent, cellSize=extent):
    arcpy.MosaicToNewRaster_management("PR_sp_beach;VI_sp_beach", env.workspace, "VIPR_sp_beach", sr, 
                                   "8_BIT_UNSIGNED", "30", "1", "SUM","FIRST")

In [30]:
# id beaches
VIPR_beach_numbers = arcpy.sa.RegionGroup("VIPR_beach", "EIGHT", "WITHIN", "ADD_LINK", None); 
VIPR_beach_numbers.save("VIPR_beach_numbers")

# calculate max spp for each beach
Beach_wSp = arcpy.ia.ZonalStatistics("VIPR_beach_numbers", "Value", "VIPR_sp_beach", "MAXIMUM", "DATA", "CURRENT_SLICE", 90, "AUTO_DETECT"); 
Beach_wSp.save("Beach_wSp")

# reclass to 2 for final raster
Beach_wSp2 = arcpy.sa.Reclassify("Beach_wSp", "Value", "1 2", "DATA"); 
Beach_wSp2.save("Beach_wSp2")

In [31]:
# reclass VIPR extent to make 0 value
# reclass to 2 for final raster
VIPR0 = arcpy.sa.Reclassify(extent, "Value", "1 0", "DATA"); 
VIPR0.save("VIPR0")

In [32]:
# beach
with arcpy.EnvManager(outputCoordinateSystem=sr, snapRaster=extent, cellSize=extent):
    arcpy.MosaicToNewRaster_management("Beach_wSp2;VIPR_beach;VIPR0", env.workspace, "Beach_Habitat", sr, 
                                   "8_BIT_UNSIGNED", "30", "1", "Maximum","FIRST")


In [33]:
# clip to 2023 VIPR Blueprint extent
VIPRMask = arcpy.sa.ExtractByMask("Beach_Habitat", extent); VIPRMask.save("VIPRMask")

## Add description and colors

In [34]:
# set code block for next step
codeblock = """
def Reclass(value):
    if value == 2:
        return '2 = Beach with 1+ nesting species predicted or observed '
    elif value == 1:
        return '1 = Other beach' 
    elif value == 0:
        return '0 = Not identified as a beach'  
"""

# add and calculate description field to hold indicator values
arcpy.management.CalculateField("VIPRMask", "descript", "Reclass(!value!)", "PYTHON3", codeblock, "TEXT")

<Result 'VIPRMask'>

In [2]:
# set code block for next step
codeblock = """
def Reclass1(Value):
	if Value == 2:
		return 234
	if Value == 1:
		return 244
	else:
		return 255
		
def Reclass2(Value):
	if Value == 2:
		return 72
	if Value == 1:
		return 151
	else:
		return 255
		
def Reclass3(Value):
	if Value == 2:
		return 59
	if Value == 1:
		return 106
	else:
		return 255
"""

# calculate Red field
arcpy.management.CalculateField("VIPRMask", "Red", "Reclass1(!Value!)", "PYTHON3", codeblock, "SHORT")
# calculate Green field
arcpy.management.CalculateField("VIPRMask", "Green", "Reclass2(!Value!)", "PYTHON3", codeblock, "SHORT")
# calculate Blue field
arcpy.management.CalculateField("VIPRMask", "Blue", "Reclass3(!Value!)", "PYTHON3", codeblock, "SHORT")

<Result 'C:\\Data\\Indicators\\Beach\\Beach.gdb\\VIPRMask'>

In [3]:
# export as .tif with VIPR extent
with arcpy.EnvManager(outputCoordinateSystem=sr, extent=Extent, snapRaster=Extent, cellSize=Extent):
    arcpy.management.CopyRaster("VIPRMask", VIPRout, '', None, "255", "NONE", "NONE", "8_BIT_UNSIGNED", "NONE", "NONE", "TIFF", "NONE", "CURRENT_SLICE", "NO_TRANSPOSE")

END